<div align="center">
<h1><img width="30" src="https://madewithml.com/static/images/rounded_logo.png">&nbsp;<a href="https://madewithml.com/">Made With ML</a></h1>
Applied ML · MLOps · Production
<br>
Join 20K+ developers in learning how to responsibly <a href="https://madewithml.com/about/">deliver value</a> with applied ML.
</div>

<br>

<div align="center">
    <a target="_blank" href="https://madewithml.com/subscribe/"><img src="https://img.shields.io/badge/Subscribe-20K-brightgreen"></a>&nbsp;
    <a target="_blank" href="https://github.com/GokuMohandas/madewithml"><img src="https://img.shields.io/github/stars/GokuMohandas/madewithml.svg?style=social&label=Star"></a>&nbsp;
    <a target="_blank" href="https://www.linkedin.com/in/goku"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>&nbsp;
    <a target="_blank" href="https://twitter.com/GokuMohandas"><img src="https://img.shields.io/twitter/follow/GokuMohandas.svg?label=Follow&style=social"></a>
    <p>🔥&nbsp; Among the <a href="https://github.com/topics/deep-learning" target="_blank">top ML</a> repositories on GitHub</p>
</div>

<br>
<hr>

# Optimize (GPU)

Use this notebooks to run hyperparameter optimization on Google Colab and utilize it's free GPUs.

In [1]:
# In case you update the code after installing
%load_ext autoreload
%autoreload 2

## Clone repository

In [2]:
# Load repository
!git clone https://github.com/GokuMohandas/applied-ml.git

Cloning into 'applied-ml'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 67 (delta 22), reused 57 (delta 16), pack-reused 0
Unpacking objects: 100% (67/67), done.


In [3]:
# Files
% cd applied-ml
!ls

/content/applied-ml
config	  LICENSE   notebooks  requirements.txt  tagifai
datasets  Makefile  README.md  setup.py


## Setup

In [ ]:
# Use latest pip
!pip install --upgrade pip

In [ ]:
# Set up
!make install-dev

## Optimize

In [6]:
from tagifai import main

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
# Download data
main.download_data()

[01/25/21 18:38:15] INFO     ✅ Data downloaded!                      main.py:46


In [8]:
# Check if data downloaded
!ls assets/data

projects.json  tags.json


In [44]:
# Optimize
main.optimize(num_trials=100)

Streaming output truncated to the last 5000 lines.
                               "hidden_dim": 336,                               
                               "dropout_p": 0.6133092831559217,                 
                               "lr": 0.00017556694564589025,                    
                               "lr_factor": 0.031775134557186994,               
                               "lr_patience": 6,                                
                               "num_epochs": 200,                               
                               "patience": 15,                                  
                               "threshold": 0.26958951354026794                 
                             }                                                  
[01/25/21 20:38:34] INFO     Epoch: 1 | train_loss: 0.00552,        train.py:144
                             val_loss: 0.00413, lr: 1.76E-04,                   
                             _patience: 15                

In [47]:
# Train best model (saving artifacts this time)
main.train_model()

[01/25/21 21:14:13] INFO     Arguments: {                           train.py:397
                               "seed": 1234,                                    
                               "cuda": true,                                    
                               "shuffle": true,                                 
                               "num_samples": 0,                                
                               "min_tag_freq": 30,                              
                               "lower": true,                                   
                               "stem": false,                                   
                               "train_size": 0.7,                               
                               "char_level": true,                              
                               "max_filter_size": 10,                           
                               "batch_size": 108,                               
                            

## Download

Download and transfer files to your local system and run the command `tagifai set-artifact-metadata` to match all metadata as if it were run from your machine.

In [51]:
from google.colab import files

In [ ]:
# Download
!zip -r best.zip assets/experiments/1
files.download('best.zip') 
files.download('assets/experiments/trials.csv')
files.download('config/args.json') 